In [1]:
## Imports
## imports
import os
import tensorflow as tf
import numpy as np
import math
import random

import scipy.misc
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#from utils import *
import pandas as pd

# Molecule data
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys 
from rdkit.Chem.Fingerprints import FingerprintMols

# Plot Data
import matplotlib.pyplot as plt

from multiprocessing.dummy import Pool as ThreadPool 

In [2]:
#Global
fps_size = 2048

In [3]:
## Activation functions

#Leaky relu
def lrelu(x, n=None, leak=0.2): 
    return tf.maximum(x, leak * x, name=n) 

#Heavside
def heavside(x, n):
    return tf.maximum(0.0,tf.sign(x), name = n)

In [4]:
## Path and parse

paths = {}
files = {}
data_sets = {}
tmp = []
f =open('Path_and_File.txt','r')
string = f.read()
list_1 = string.split('##')


[tmp.append(x.split('=')[0].rstrip()) for x in list_1[1].split('#')[1:]]
[paths.update({i : []} )for i in tmp]
[paths[x.split('=')[0].rstrip()].append(x.split('= ')[1].rstrip()) for x in list_1[1].split('#')[1:]]
tmp=[]
[tmp.append(x.split('=')[0].rstrip()) for x in list_1[2].split('#')[1:]]
[files.update({i : []} )for i in tmp]
[files[x.split('=')[0].rstrip()].append(x.split('= ')[1].rstrip()) for x in list_1[2].split('#')[1:]]
tmp = []
[tmp.append(x.split('=')[0].rstrip()) for x in list_1[3].split('#')[1:]]
[data_sets.update({i : []} )for i in tmp]
[data_sets[x.split('=')[0].rstrip()].append(x.split('= ')[1].rstrip()) for x in list_1[3].split('#')[1:]]
f.close()


In [ ]:
## Data extraction

def morgan_fps(mol):
    fps_morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 6, nBits=fps_size)
    fps_morgan = np.array(list(fps_morgan.ToBitString())).astype('float32')
    #print('Morgan', (sum(fps_morgan)/fps_size))
    return fps_morgan
    
def maccs_fps(mol):
    fps_maccs = MACCSkeys.GenMACCSKeys(mol)#, nBits =1024)
    return np.array(list(fps_maccs.ToBitString())).astype('float32')

def topological_fps(mol):
    fps_top = FingerprintMols.FingerprintMol(mol, minPath=1, 
                                                maxPath=7, 
                                                fpSize=fps_size,
                                                bitsPerHash=2,
                                                useHs=False,
                                                tgtDensity = 0.0,
                                                minSize = 128 
                                                )
    fps_top = np.array(list(fps_top.ToBitString())).astype('float32')
    #print('Top', (sum(fps_top)/fps_size))
    return fps_top
    
def process_data_np(ds_name = 'LD50_Test_data', pool_size = 4):
    

    
    data_dir = paths['Path to training set'][0]
    df = pd.read_csv(os.path.join(data_dir, ds_name + '.csv'))
    pool = ThreadPool(pool_size)
    fps_morgan, fps_maccs, fps_top, mol = [], [], [], []
    fps_dict = {}
    l = np.array(df.value, dtype = np.float32)

    lable_scale = np.arange(int(min(l)), int(max(l)),int((max(l)-min(l))/10))
    for i in range(len(lable_scale)-1):
        if(i < len(lable_scale)-2):
            l[np.logical_and(np.greater_equal(l, lable_scale[i]),np.less_equal(l, lable_scale[i+1]))] = i
        else:
            l[(np.greater_equal(l, lable_scale[i]))] = i
    labels = [np.zeros([10], dtype = np.float32) for i in range(len(l))]
    for i in range(len(l)):
        labels[i][int(l[i])] = 1
        
    mol = pool.map(Chem.MolFromSmiles, list(df.smiles))

    fps_morgan = pool.map(morgan_fps, mol)
    fps_top = pool.map(topological_fps, mol)
    fps = np.array([fps_morgan, fps_top], dtype = np.float32)
    fps = np.reshape(fps,[len(mol),fps_size,2])
    fps =fps - (np.equal(fps,np.zeros(np.shape(fps)))) # from [0,1] -> [-1,1]
    
    fps_dict['fps_train_np'], fps_dict['fps_test_np'], fps_dict['labels_train_np'], fps_dict['labels_test_np'] = train_test_split(fps,
                                                                      labels,
                                                                      test_size=0.20,
                                                                      random_state=42)
    
    fps_dict['fps_train_list'] = [tf.convert_to_tensor(x, dtype = tf.float32) for x in fps_dict['fps_train_np']]
    fps_dict['fps_test_list'] = [tf.convert_to_tensor(x, dtype = tf.float32) for x in fps_dict['fps_test_np']]
    fps_dict['labels_train_list'] = [tf.convert_to_tensor(x, dtype = tf.float32) for x in fps_dict['labels_train_np']]
    fps_dict['labels_test_list'] = [tf.convert_to_tensor(x, dtype = tf.float32) for x in fps_dict['labels_test_np']]

    
    return fps_dict

In [ ]:
data_dir = paths['Path to training set'][0]
ds_name = 'LD50_Test_data'
df = pd.read_csv(os.path.join(data_dir, ds_name + '.csv'))

l = np.array(df.value)

label_scale = np.arange(int(min(l)), int(max(l)),int((max(l)-min(l))/10))
for i in range(len(label_scale)-1):
    if(i < len(label_scale)-2):
        l[np.logical_and(np.greater_equal(l, label_scale[i]),np.less_equal(l, label_scale[i+1]))] = i
    else:
        l[(np.greater_equal(l, label_scale[i]))] = i
labels = np.zeros([len(l), 10])
for i in range(len(l)):
    labels[i, int(l[i])] = 1


In [ ]:
## Data processing with numpy
def batch_gen(fps, label, batch_size = 32, FLAG = True):   
    
    num_batch_epoch = math.floor(len(fps)/batch_size) 
    fps_batch_list = []
    label_batch_list = []
    if FLAG:
        batch_index = np.random.randint(0, len(label), (batch_size, num_batch_epoch))  
    else:
        batch_index = np.arange(batch_size*num_batch_epoch).reshape((batch_size,num_batch_epoch))

    for i in range(num_batch_epoch):
   
        tmp1, tmp2 = [], []
        tmp1 = [fps[x] for x in batch_index[:,i]]
        tmp2 = [label[x] for x in batch_index[:,i]]
        fps_batch_list.append(tmp1)
        label_batch_list.append(tmp2)
    batch={'fps': fps_batch_list, 'label': label_batch_list}
    return batch

In [ ]:
## Encoder 

def encoder(input, fps_dim, layers_dim, is_train_enc, reuse=False):

    with tf.variable_scope('encode') as scope:
        if reuse:
            scope.reuse_variables()
        
        #Convolution module M
        out_cnn1 = tf.layers.conv1d(inputs = input, filters = 32, kernel_size = 7,
                        strides = 4, padding = 'valid', data_format ='channels_last',
                        dilation_rate = 1, activation = 'relu',
                        use_bias = True, kernel_initializer = tf.truncated_normal_initializer(stddev=0.02),
                        bias_initializer = tf.constant_initializer(0.0),
                        kernel_regularizer = tf.keras.regularizers.l2(l=0.01),
                        name='encoder_cnn1', reuse = reuse)
        
        
        out_cnn2 = tf.layers.conv1d(inputs = out_cnn1, filters = 32, kernel_size = 7,
                        strides = 4, padding = 'valid', data_format ='channels_last',
                        dilation_rate = 1, activation = 'relu',
                        use_bias = True, kernel_initializer = tf.truncated_normal_initializer(stddev=0.02),
                        bias_initializer = tf.constant_initializer(0.0),
                        kernel_regularizer = tf.keras.regularizers.l2(l=0.01),
                        name='encoder_cnn2', reuse = reuse)
        
        out_dpu1 = tf.layers.dropout(out_cnn2,
                        rate=0.3,
                        noise_shape=None,
                        seed=None,
                        training=False,
                        name=None)
        
        out_cnn3 = tf.layers.conv1d(inputs = out_dpu1, filters = 64, kernel_size = 5,
                        strides = 3, padding = 'valid', data_format ='channels_last',
                        dilation_rate = 1, activation = 'relu',
                        use_bias = True, kernel_initializer = tf.truncated_normal_initializer(stddev=0.02),
                        bias_initializer = tf.constant_initializer(0.0),
                        kernel_regularizer = tf.keras.regularizers.l2(l=0.01),
                        name='encoder_cnn3', reuse = reuse)
        
        out_cnn4 = tf.layers.conv1d(inputs = out_cnn3, filters = 64, kernel_size = 5,
                        strides = 3, padding = 'valid', data_format ='channels_last',
                        dilation_rate = 1, activation = 'relu',
                        use_bias = True, kernel_initializer = tf.truncated_normal_initializer(stddev=0.02),
                        bias_initializer = tf.constant_initializer(0.0),
                        kernel_regularizer = tf.keras.regularizers.l2(l=0.01),
                        name='encoder_cnn4', reuse = reuse)
        
        out_dpu2 = tf.layers.dropout(out_cnn4,
                        rate=0.3,
                        noise_shape=None,
                        seed=None,
                        training=False,
                        name=None)

        out_bn1 = tf.layers.batch_normalization(out_dpu2)
        #Flattening 
        out_flat = tf.layers.flatten(out_bn1, name = 'encode_flatten')
        

        # Input layer (layers_dim[0]) encoding layer
        dense0 = tf.layers.dense(out_flat,40,
                                activation='relu',
                                use_bias=True,
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                bias_initializer=tf.zeros_initializer(),
                                name ='first_dense',
                                reuse=reuse)
        bn0 = tf.contrib.layers.batch_norm(dense0, is_training = is_train_enc, epsilon=1e-5,
                                           decay = 0.9,updates_collections=None, scope = 'bn0')
        # First hidedn layer (layers_dim[1]) encoding layer
        w1 = tf.get_variable('w2', shape=[40, 10], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b2', shape=[10], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense1 = lrelu(tf.add(tf.matmul(bn0, w1), b1, name='mat_mul_2'), 'encoder_hlayer', leak = 0.2)
        bn1 = tf.contrib.layers.batch_norm(dense1, is_training = is_train_enc, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn1')
        bn1 = tf.layers.batch_normalization(bn1)
        sizes_encoder = {}
        sizes_encoder['enc_input'] = input.get_shape().as_list()
        sizes_encoder['cnn1_out'] = out_cnn1.get_shape().as_list()
        sizes_encoder['cnn2_out'] = out_cnn2.get_shape().as_list()
        sizes_encoder['dpu1_out'] = out_dpu1.get_shape().as_list()
        sizes_encoder['cnn3_out'] = out_cnn3.get_shape().as_list()
        sizes_encoder['cnn4_out'] = out_cnn4.get_shape().as_list()
        sizes_encoder['dpu2_out'] = out_dpu2.get_shape().as_list()
        sizes_encoder['flatten'] = out_flat.get_shape().as_list()
        sizes_encoder['bn0'] = bn0.get_shape().as_list()
        sizes_encoder['bn1'] = bn1.get_shape().as_list()
                                                        
        return bn1, sizes_encoder

In [ ]:
## Decoder

def decoder(input, fps_dim, layers_dim,batch_size, is_train_dec=True, reuse=False):
    with tf.variable_scope('decode') as scope:
        if reuse:
            scope.reuse_variables()
            
        # Input layer (layers_dim[0]) encoding layer
        
        dense2 = tf.layers.dense(input,layers_dim['bn0'][1], activation='relu', use_bias=True, kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                bias_initializer=tf.zeros_initializer(), name ='first_dense', reuse=reuse)
        bn2 = tf.contrib.layers.batch_norm(dense2, is_training = is_train_dec, epsilon=1e-5,
                                           decay = 0.9,updates_collections=None, scope = 'bn0')
        # First hidedn layer (layers_dim[1]) encoding layer
        w3 = tf.get_variable('w3', shape=[layers_dim['bn0'][1], layers_dim['flatten'][1]], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b3 = tf.get_variable('b3', shape=[layers_dim['flatten'][1]], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense3 = lrelu(tf.add(tf.matmul(bn2, w3), b3, name='mat_mul_3'), 'latent_out', leak = 0.2)
        bn3 = tf.contrib.layers.batch_norm(dense3, is_training = is_train_dec, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn1')
        bn3 = tf.layers.batch_normalization(bn3)  
        bn3 = tf.reshape(bn3,[-1,layers_dim['cnn4_out'][1],layers_dim['cnn4_out'][2]], name ='decode_reshape')
        #Convolution module M'
        out_dpu3 = tf.layers.dropout(bn3,rate=0.3,name= 'decoder_droput_1')
        
        l1 = out_dpu3.get_shape()
        filter_1 = tf.get_variable('Decoder_filters_1', shape=[5, 64, 64], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02)) 
        out_cnn5 = lrelu(tf.contrib.nn.conv1d_transpose(out_dpu3,filter_1 ,
                                                        output_shape= [batch_size,layers_dim['cnn3_out'][1],64],
                                                        stride = 3,
                                                        padding='VALID',data_format='NWC',
                                                        name='deconvolution_1'), leak= 0.0)
        
        filter_2 = tf.get_variable('Decoder_filters_2', shape=[5, 32, 64], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02)) 
        out_cnn6 = lrelu(tf.contrib.nn.conv1d_transpose(value = out_cnn5,filter = filter_2,
                                                        output_shape= [batch_size,layers_dim['cnn2_out'][1],32],
                                                        stride = 3, padding='VALID',name='deconvolution_2'), leak=0.0)
        
        out_dpu4 = tf.layers.dropout(out_cnn6, rate=0.3, name='Decoder_droput_2')

        filter_3 = tf.get_variable('Decoder_filters_3', shape=[7, 32, 32], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02)) 
        out_cnn7 = lrelu(tf.contrib.nn.conv1d_transpose(value = out_cnn6,filter = filter_3,
                                                        output_shape= [batch_size,layers_dim['cnn1_out'][1],32],stride = 4,
                                                        padding='VALID',name='deconvolution_3'),
                                                         leak=0.0)
        l4 = tf.shape(out_dpu3)[0]
        filter_4 = tf.get_variable('Decoder_filters_4', shape=[7, 2, 32], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02)) 
        out_cnn8 = lrelu(tf.contrib.nn.conv1d_transpose(
                        value = out_cnn7,
                        filter = filter_4,
                        output_shape= [batch_size,layers_dim['enc_input'][1],layers_dim['enc_input'][2]],
                        stride = 4,
                        padding='VALID',
                        data_format='NWC',
                        name='deconvolution_4'), leak=0.0)

        out_bn4 = tf.layers.batch_normalization(out_cnn8)

        #Flattening 
        decoded_fingerprint = tf.nn.tanh(out_bn4, name = 'squash') 
        
        return decoded_fingerprint

In [ ]:
## Discriminator

def discriminator(input,layers_dim, is_train_dis = True, reuse=False):
    
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()
                
        # Input layer (layers_dim[0]) encoding layer
        w1 = tf.get_variable('w1_dis', shape=[layers_dim[2], layers_dim[2]-3], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1_dis', shape=[layers_dim[2]-3], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense1 = lrelu(tf.add(tf.matmul(input, w1), b1, name='dis_mat_mul_1'), 'discriminator_input_layer', leak = 0.2)
        bn1 = tf.contrib.layers.batch_norm(dense1, is_training = is_train_dis, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn1_dis')  

        # First hidedn layer (layers_dim[1]) encoding layer
        w2 = tf.get_variable('w2_dis', shape=[layers_dim[2]-3, 5], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2_dis', shape=[5], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        dense2 = lrelu(tf.add(tf.matmul(bn1, w2), b2, name='dis_mat_mul_2'), 'encoder_hlayer', leak = 0.2)
        bn2 = tf.contrib.layers.batch_norm(dense2, is_training = is_train_dis, epsilon=1e-5, decay = 0.9,
                                           updates_collections=None, scope = 'bn2_dis')


        w3 = tf.get_variable('w3_dis', shape=[5, 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b3 = tf.get_variable('b3_dis', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        # wgan just get rid of the sigmoid
        logits = tf.add(tf.matmul(bn2, w3), b3, name='logits')
        # dcgan
        acted_out = tf.nn.sigmoid(logits)
        return logits #, acted_out

In [ ]:
## Trainer

def train(FLAG_label):
    
    FLAG = True
    save_cluster = True
    continue_training = False
    save_model=False
    data_set = 'LD50_tmp'
    if FLAG_label:
        path = '/mnt/SSD1/data/LD50/LD50_training_labeld_data/'
    else:
        path = '/mnt/SSD1/data/LD50/LD50_training_data/'
    file_name = files['Training data file name'][0] + data_set
    batch_size = 64
    fps_dim = 2048
    if FLAG_label:
        layers_dim = np.array([256, 64, 20])
    else:
        layers_dim = np.array([256,64,10])
    fps_dict = process_data_np()
    fps_train = fps_dict['fps_train_np']
    fps_test = fps_dict['fps_test_list']
    labels_train_np = fps_dict['labels_train_np']
    labels_train = fps_dict['labels_train_list']
    labels_test = fps_dict['labels_test_np']
    
    real = np.ones([batch_size,1])
    fake = np.zeros([batch_size,1])
    
    val_lr_enc = 0.00002
    val_lr_dis = 0.00002
    val_lr_dec = 0.00002
    
    with tf.variable_scope('input'):
        #real and fake image placholders
        real_fps = tf.placeholder(tf.float32, shape = [None, fps_dim,2], name='real_fps')
        
        dist_encode = tf.placeholder(tf.float32, shape=[None, layers_dim[2]], name='gen_fps')
        real_latent = tf.placeholder(tf.float32, shape=[None, 1], name='real_latent')
        fake_latent = tf.placeholder(tf.float32, shape=[None, 1], name ='fake_latent')
        
        label = tf.placeholder(tf.float32, shape= [None, 10], name='label')
        
        is_train_enc = tf.placeholder(tf.bool, name='is_train_enc')
        is_train_dis = tf.placeholder(tf.bool, name = 'is_train_dis')
        is_train_dec = tf.placeholder(tf.bool, name = 'is_train_dec')
        
        lr_enc = tf.placeholder(tf.float64, name = 'learning_rate_encoder')
        lr_dis = tf.placeholder(tf.float64, name = 'learning_rate_discriminator')
        lr_dec = tf.placeholder(tf.float64, name = 'learning_rate_decoder')
        
    # wgan
    real_encode, layers_dim_enc = encoder(real_fps, fps_dim, layers_dim, is_train_enc, reuse=False)
    real_decode = decoder(real_encode, fps_dim, layers_dim_enc,batch_size, is_train_dec, reuse= False)
    
    real_result = discriminator(dist_encode, layers_dim, is_train_dis, reuse =False)
    if FLAG_label:
        x = tf.concat([real_encode, label], 1)        
    else:
        x = real_encode
    fake_result = discriminator(x, layers_dim, is_train_dis, reuse=True)
    
    dis_loss_real = tf.losses.mean_squared_error(real_result, real_latent)
    dis_loss_fake = tf.losses.mean_squared_error(fake_result, fake_latent)
    dis_loss = tf.reduce_mean(dis_loss_real) + tf.reduce_mean(dis_loss_fake)
    #dis_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  # This optimizes the discriminator.
    enc_loss = -tf.reduce_mean(tf.losses.mean_squared_error(fake_result, fake_latent))  # This optimizes the generator.
    #dec_loss = tf.reduce_mean(tf.square(real_fps - real_decode))        
    dec_loss = tf.losses.mean_squared_error(real_fps, real_decode)
    t_vars = tf.trainable_variables()
    
    dis_vars = [var for var in t_vars if 'dis' in var.name]
    enc_vars = [var for var in t_vars if 'encode' in var.name]
    dec_vars = [var for var in t_vars if 'decode' in var.name]
    
    trainer_dis = tf.train.AdamOptimizer(learning_rate =lr_dis, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_discriminator').minimize(dis_loss,var_list=dis_vars)
    trainer_dec = tf.train.AdamOptimizer(learning_rate =lr_dec, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_decoder').minimize(dec_loss, var_list=dec_vars)
    trainer_enc = tf.train.AdamOptimizer(learning_rate =lr_enc, beta1=0.9, beta2=0.999,
                                         epsilon=1e-08, use_locking=False,
                                         name='Adam_encoder').minimize(enc_loss, var_list=enc_vars)


    
    
    
    
    
    total_batch = 0
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    # continue training
    if continue_training:
        save_path = saver.save(sess, "/mnt/HDD1/models/LD50/model.ckpt")
        ckpt = tf.train.latest_checkpoint('/mnt/HDD1/models/LD50')
        saver.restore(sess, save_path)

    gpu_options = tf.GPUOptions(allow_growth=True)
    session_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False,
                                    gpu_options=gpu_options)
    #threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    num_batch_epoch = math.floor(len(fps_train)/batch_size)
    print('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, num_batch_epoch, 5000))
    print('start training...')
    for i in range(2001):
        batch = batch_gen(fps_train, labels_train_np, batch_size = batch_size, FLAG=FLAG)
        train_fps = batch['fps']
        train_label = batch['label']
        disLoss = np.zeros([50])
        encLoss = np.zeros([50])
        decLoss = np.zeros([50])
        for j in range(num_batch_epoch):
            
            iters = [5, 1, 1]
            if FLAG_label:
                index = np.nonzero(train_label[j])[1]
                train_noise = [np.concatenate((np.random.normal(loc = x, scale = 0.5,size=[layers_dim[2]-10]).astype(np.float32), train_label[j][num])) for num, x in enumerate(index)]
                #train_noise = np.concatenate((np.array(train_noise), train_label[j]), axis = 1) 
                train_noise = np.array(train_noise)
            else:
                train_noise = np.random.normal(loc = 0,scale = 0.5,size=[batch_size, layers_dim[2]]).astype(np.float32)
            # Update the discriminator
            for k in range(iters[0]): 
                dict_dis = {real_fps: train_fps[j], dist_encode: train_noise, label: train_label[j],lr_dis: val_lr_dis,
                            real_latent: real, fake_latent: fake, is_train_dis: True, is_train_enc: False}
                sess.run([trainer_dis], feed_dict=dict_dis)
            # Update the encoder
            for k in range(iters[1]):
                dict_enc = {real_fps: train_fps[j], label: train_label[j], lr_enc: val_lr_enc, real_latent: real,
                            fake_latent: fake, is_train_enc: True, is_train_dis: False}
                sess.run([trainer_enc], feed_dict=dict_enc)

            for k in range(iters[2]):
                dict_dec = {real_fps: train_fps[j],lr_dec: val_lr_dec,real_latent: real,
                            fake_latent: fake,is_train_enc: False, is_train_dec: True}
                sess.run([trainer_dec], feed_dict=dict_dec)
                
            dict_loss = {real_fps: train_fps[j], dist_encode: train_noise,label: train_label[j], real_latent: real,
                         fake_latent: fake, is_train_dis: False, is_train_enc: False, is_train_dec: False,}
            disLoss[i%50], encLoss[i%50], decLoss[i%50] = sess.run([dis_loss, enc_loss, dec_loss],feed_dict = dict_loss ) 
        if(i == 0):
            avg_disLoss = disLoss[i]
            avg_encLoss = encLoss[i]
            avg_decLoss = decLoss[i]
        
        # Uppdating learning rate
        if i%49 == 0 and i != 0:
            if (abs(avg_disLoss - (sum(disLoss)/50)) < (0.02*avg_disLoss)):
                 val_lr_dis = val_lr_dis * 0.2
            if (abs(avg_encLoss - (sum(encLoss)/50)) < (0.02*avg_encLoss)):
                 val_lr_enc = val_lr_enc * 0.2
            if (abs(avg_decLoss - (sum(decLoss)/50)) < (0.01*avg_decLoss)):
                 val_lr_dec = val_lr_dec * 0.2
            avg_disLoss = sum(disLoss)/50
            avg_encLoss = sum(encLoss)/50
            avg_decLoss = sum(decLoss)/50
                 
        # save check point every 500 epoch
        if i%500 == 0 and i != 0:
            if save_model:
                if not os.path.exists('/mnt/HDD1/models/LD50/' ):
                    os.makedirs('/mnt/HDD1/models/LD50/')
                saver.save(sess, '/mnt/HDD1/models/LD50/' + 'model' + str(i) + '.ckpt') 
        # Saving cluster data
        if i%10 == 0 and save_cluster :
            
            FLAG = False
            batch = batch_gen(fps_train, labels_train_np, batch_size = batch_size, FLAG=FLAG)
            FLAG = True
            train_fps = batch['fps']
            train_label = batch['label']
            pca = PCA(n_components=2)
            # Get lables in np array form
            label_array_list = fps_dict['labels_train_np']
            latent_vectors_list=[]
            for j in range(num_batch_epoch):
                dict_enc = {real_fps: train_fps[j], is_train_enc: False}
                latent_vectors_list.append(sess.run([real_encode], feed_dict = dict_enc ))

            latent_vectors = np.reshape(np.array(latent_vectors_list), (len(np.array(latent_vectors_list))*batch_size,10))
            features = ['Node0', 'Node1', 'Node2', 'Node3', 'Node4', 'Node5', 'Node6', 'Node7', 'Node8', 'Node9']
            df = pd.DataFrame(latent_vectors, columns=features)
           
            df['label'] = np.nonzero(np.array(label_array_list[:num_batch_epoch*batch_size]))[1]
            # Separating out the features
            x = df.loc[:, features[:-1]].values

            # Separating out the target
            y = df.loc[:,['label']].values

            # Standardizing the features
            #if not FLAG_label:
             #   x = StandardScaler().fit_transform(x)
            principalComponents = pca.fit_transform(x)

            principalDf = pd.DataFrame(data = principalComponents,
                                       columns = ['principal component 1', 'principal component 2'])
            finalDf = pd.concat([df['Node0'],df['Node1'], df[['label']]], axis = 1)
            if FLAG_label:
                if not os.path.exists('/mnt/SSD1/data/cluster_label/LD50_tmp/' ):
                    os.makedirs('/mnt/SSD1/data/cluster_label/LD50_tmp/') 
                finalDf.to_csv('/mnt/SSD1/data/cluster_label/LD50_tmp/' + 'cluster_data_epoch_' + str(i) + '.csv')
            else:
                if not os.path.exists('/mnt/SSD1/data/cluster/LD50/' ):
                    os.makedirs('/mnt/SSD1/data/cluster/LD50/') 
                finalDf.to_csv('/mnt/SSD1/data/cluster/LD50/' + 'cluster_data_epoch_' + str(i) + '.csv')
                
        #Save training data 
        if i%5 == 0:
            if not os.path.exists(path):
                os.makedirs(path)
            if not os.path.isfile(path + file_name) or i == 0:
                fo = open(path +file_name, 'w+')
            else:
                fo = open(path + file_name,'a')
             
            
            fo.write('%d;%f;%f;%f\n' % (i, disLoss[i%50],  encLoss[i%50],  decLoss[i%50]))
            fo.close()

        print('Label='  ,FLAG_label,i, 'disLoss = ',disLoss[i%50], 'encLoss = ',encLoss[i%50], 'decLoss = ', decLoss[i%50] )


In [ ]:
## clustering



In [ ]:
## similarity test 
def sim_test():
    random_dim 
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    
    # wgan
    fake_image = generator(random_input, random_dim, is_train)
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    variables_to_restore = slim.get_variables_to_restore(include=['gen'])
    print(variables_to_restore)
    saver = tf.train.Saver(variables_to_restore)
    ckpt = tf.train.latest_checkpoint('./model/' + version)
    saver.restore(sess, ckpt)


In [ ]:
## Main

if __name__ == "__main__":
    
    train(FLAG_label = True)

batch size: 64, batch num per epoch: 103, epoch num: 5000
start training...
Label= True 0 disLoss =  0.986304759979248 encLoss =  -0.00022554387396667153 decLoss =  0.9972693920135498
Label= True 1 disLoss =  0.9670835733413696 encLoss =  -0.0007712544174864888 decLoss =  0.9945712089538574
Label= True 2 disLoss =  0.9351528882980347 encLoss =  -0.00297438632696867 decLoss =  0.991916298866272
Label= True 3 disLoss =  0.9209959506988525 encLoss =  -0.005643478594720364 decLoss =  0.9907875061035156
Label= True 4 disLoss =  0.88201504945755 encLoss =  -0.008448632434010506 decLoss =  0.9876953363418579
Label= True 5 disLoss =  0.8501888513565063 encLoss =  -0.012539723888039589 decLoss =  0.9848047494888306
Label= True 6 disLoss =  0.8058760166168213 encLoss =  -0.01756194606423378 decLoss =  0.9832503199577332
Label= True 7 disLoss =  0.7876691222190857 encLoss =  -0.025595083832740784 decLoss =  0.9806236028671265
Label= True 8 disLoss =  0.75254225730896 encLoss =  -0.031634252518415

Label= True 77 disLoss =  0.4958871006965637 encLoss =  -0.24674172699451447 decLoss =  0.821523129940033
Label= True 78 disLoss =  0.49279382824897766 encLoss =  -0.24438175559043884 decLoss =  0.8346759080886841
Label= True 79 disLoss =  0.49942633509635925 encLoss =  -0.24661362171173096 decLoss =  0.8160704374313354
Label= True 80 disLoss =  0.4910537004470825 encLoss =  -0.24447095394134521 decLoss =  0.8083948493003845
Label= True 81 disLoss =  0.4963882863521576 encLoss =  -0.24680134654045105 decLoss =  0.8275244832038879
Label= True 82 disLoss =  0.4973633885383606 encLoss =  -0.24689197540283203 decLoss =  0.8359841704368591
Label= True 83 disLoss =  0.49498265981674194 encLoss =  -0.2447504848241806 decLoss =  0.7921265959739685
Label= True 84 disLoss =  0.49071067571640015 encLoss =  -0.24303968250751495 decLoss =  0.7997760772705078
Label= True 85 disLoss =  0.4912894368171692 encLoss =  -0.24590280652046204 decLoss =  0.8216491937637329
Label= True 86 disLoss =  0.4866306

Label= True 154 disLoss =  0.4967043995857239 encLoss =  -0.24792052805423737 decLoss =  0.7032644748687744
Label= True 155 disLoss =  0.5008220672607422 encLoss =  -0.24862423539161682 decLoss =  0.7163013815879822
Label= True 156 disLoss =  0.5012141466140747 encLoss =  -0.2488558292388916 decLoss =  0.7296910881996155
Label= True 157 disLoss =  0.5002476572990417 encLoss =  -0.24944904446601868 decLoss =  0.7007634043693542
Label= True 158 disLoss =  0.49543339014053345 encLoss =  -0.245401531457901 decLoss =  0.7104483246803284
Label= True 159 disLoss =  0.5016589164733887 encLoss =  -0.24882668256759644 decLoss =  0.6851967573165894
Label= True 160 disLoss =  0.49856042861938477 encLoss =  -0.24807941913604736 decLoss =  0.7023115754127502
Label= True 161 disLoss =  0.5091894865036011 encLoss =  -0.25145137310028076 decLoss =  0.6773799061775208
Label= True 162 disLoss =  0.5029357671737671 encLoss =  -0.24885086715221405 decLoss =  0.6786025762557983
Label= True 163 disLoss =  0.

Label= True 231 disLoss =  0.5011351704597473 encLoss =  -0.24858754873275757 decLoss =  0.6132649183273315
Label= True 232 disLoss =  0.49902474880218506 encLoss =  -0.24775268137454987 decLoss =  0.6314941644668579
Label= True 233 disLoss =  0.5025463104248047 encLoss =  -0.24923387169837952 decLoss =  0.6117047071456909
Label= True 234 disLoss =  0.5075064301490784 encLoss =  -0.25142818689346313 decLoss =  0.6341769695281982
Label= True 235 disLoss =  0.5000575184822083 encLoss =  -0.24800114333629608 decLoss =  0.6219592094421387
Label= True 236 disLoss =  0.5039453506469727 encLoss =  -0.2498195469379425 decLoss =  0.6066346168518066
Label= True 237 disLoss =  0.5051290988922119 encLoss =  -0.2503173053264618 decLoss =  0.6274463534355164
Label= True 238 disLoss =  0.502887487411499 encLoss =  -0.24967187643051147 decLoss =  0.6273698210716248
Label= True 239 disLoss =  0.49859535694122314 encLoss =  -0.24863925576210022 decLoss =  0.6068305373191833
Label= True 240 disLoss =  0.

Label= True 308 disLoss =  0.5058987140655518 encLoss =  -0.250938355922699 decLoss =  0.5792456865310669
Label= True 309 disLoss =  0.5067540407180786 encLoss =  -0.25089994072914124 decLoss =  0.5449295043945312
Label= True 310 disLoss =  0.5031978487968445 encLoss =  -0.250113308429718 decLoss =  0.5267833471298218
Label= True 311 disLoss =  0.500346839427948 encLoss =  -0.24811941385269165 decLoss =  0.6441629528999329
Label= True 312 disLoss =  0.502916693687439 encLoss =  -0.24870043992996216 decLoss =  0.603195071220398
Label= True 313 disLoss =  0.5088073015213013 encLoss =  -0.25146085023880005 decLoss =  0.5693551301956177
Label= True 314 disLoss =  0.5036545991897583 encLoss =  -0.24998244643211365 decLoss =  0.6522819399833679
Label= True 315 disLoss =  0.5038928389549255 encLoss =  -0.24864919483661652 decLoss =  0.5534286499023438
Label= True 316 disLoss =  0.5068401098251343 encLoss =  -0.2502819895744324 decLoss =  0.5519694089889526
Label= True 317 disLoss =  0.5017156

...

Label= True 1153 disLoss =  0.558885931968689 encLoss =  -0.27783408761024475 decLoss =  0.5874934196472168
Label= True 1154 disLoss =  0.5492106080055237 encLoss =  -0.2675073742866516 decLoss =  0.4707392454147339
Label= True 1155 disLoss =  0.5562398433685303 encLoss =  -0.2795857787132263 decLoss =  0.5080019235610962
Label= True 1156 disLoss =  0.5726442337036133 encLoss =  -0.28689494729042053 decLoss =  0.5338331460952759
Label= True 1157 disLoss =  0.5603247284889221 encLoss =  -0.27437496185302734 decLoss =  0.510842502117157
Label= True 1158 disLoss =  0.5543988347053528 encLoss =  -0.2787296772003174 decLoss =  0.4504166841506958
Label= True 1159 disLoss =  0.5677964091300964 encLoss =  -0.2815858721733093 decLoss =  0.552088737487793
Label= True 1160 disLoss =  0.5621124505996704 encLoss =  -0.2765268087387085 decLoss =  0.6046494841575623
Label= True 1161 disLoss =  0.5605210065841675 encLoss =  -0.2746795415878296 decLoss =  0.578765869140625
Label= True 1162 disLoss =  0

Label= True 1229 disLoss =  0.5533557534217834 encLoss =  -0.2836979627609253 decLoss =  0.519554853439331
Label= True 1230 disLoss =  0.5617742538452148 encLoss =  -0.2937113344669342 decLoss =  0.6305902004241943
Label= True 1231 disLoss =  0.5596193075180054 encLoss =  -0.2786406874656677 decLoss =  0.48180463910102844
Label= True 1232 disLoss =  0.5665512084960938 encLoss =  -0.28490149974823 decLoss =  0.543632984161377
Label= True 1233 disLoss =  0.5672332048416138 encLoss =  -0.27874046564102173 decLoss =  0.5890636444091797
Label= True 1234 disLoss =  0.5550535917282104 encLoss =  -0.27306991815567017 decLoss =  0.5677189826965332
Label= True 1235 disLoss =  0.5623180866241455 encLoss =  -0.2762078642845154 decLoss =  0.44111180305480957
Label= True 1236 disLoss =  0.556044340133667 encLoss =  -0.2756379544734955 decLoss =  0.6658473014831543
Label= True 1237 disLoss =  0.5635737180709839 encLoss =  -0.28270697593688965 decLoss =  0.5393942594528198
Label= True 1238 disLoss =  

Label= True 1305 disLoss =  0.5525829792022705 encLoss =  -0.27171388268470764 decLoss =  0.6167182326316833
Label= True 1306 disLoss =  0.565438985824585 encLoss =  -0.2792792022228241 decLoss =  0.5946004986763
Label= True 1307 disLoss =  0.5553726553916931 encLoss =  -0.2823036313056946 decLoss =  0.49973708391189575
Label= True 1308 disLoss =  0.5706663131713867 encLoss =  -0.28162628412246704 decLoss =  0.583105742931366
Label= True 1309 disLoss =  0.5579522252082825 encLoss =  -0.2799307703971863 decLoss =  0.5903099775314331
Label= True 1310 disLoss =  0.567080020904541 encLoss =  -0.29875272512435913 decLoss =  0.5477839112281799
Label= True 1311 disLoss =  0.5543488264083862 encLoss =  -0.28334763646125793 decLoss =  0.5748183727264404
Label= True 1312 disLoss =  0.5764889121055603 encLoss =  -0.2837063670158386 decLoss =  0.5638078451156616
Label= True 1313 disLoss =  0.5602273941040039 encLoss =  -0.27974751591682434 decLoss =  0.4873998165130615
Label= True 1314 disLoss =  

Label= True 1381 disLoss =  0.5595678091049194 encLoss =  -0.28089869022369385 decLoss =  0.5456447601318359
Label= True 1382 disLoss =  0.5612411499023438 encLoss =  -0.2716027498245239 decLoss =  0.5190310478210449
Label= True 1383 disLoss =  0.5723443031311035 encLoss =  -0.27849411964416504 decLoss =  0.48720139265060425
Label= True 1384 disLoss =  0.571087658405304 encLoss =  -0.2800164222717285 decLoss =  0.5295902490615845
Label= True 1385 disLoss =  0.5671607255935669 encLoss =  -0.28696465492248535 decLoss =  0.5575928092002869
Label= True 1386 disLoss =  0.573104977607727 encLoss =  -0.2839866876602173 decLoss =  0.5806607007980347
Label= True 1387 disLoss =  0.5702764987945557 encLoss =  -0.2755461633205414 decLoss =  0.505963146686554
Label= True 1388 disLoss =  0.565805971622467 encLoss =  -0.27672266960144043 decLoss =  0.6458885073661804
Label= True 1389 disLoss =  0.5603234171867371 encLoss =  -0.27529215812683105 decLoss =  0.6492899656295776
Label= True 1390 disLoss =

Label= True 1457 disLoss =  0.5813311338424683 encLoss =  -0.3009062707424164 decLoss =  0.5519987344741821
Label= True 1458 disLoss =  0.5607632398605347 encLoss =  -0.2827151417732239 decLoss =  0.4901735782623291
Label= True 1459 disLoss =  0.5569764971733093 encLoss =  -0.2795864939689636 decLoss =  0.46058952808380127
Label= True 1460 disLoss =  0.5944018363952637 encLoss =  -0.3105270266532898 decLoss =  0.5642937421798706
Label= True 1461 disLoss =  0.5810145139694214 encLoss =  -0.28896501660346985 decLoss =  0.5947256088256836
Label= True 1462 disLoss =  0.5730416178703308 encLoss =  -0.28544509410858154 decLoss =  0.48007938265800476
Label= True 1463 disLoss =  0.5763412714004517 encLoss =  -0.30010905861854553 decLoss =  0.5827395915985107
Label= True 1464 disLoss =  0.5538070201873779 encLoss =  -0.27450311183929443 decLoss =  0.5041220188140869


In [ ]:
print(51%50)